In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

## Pandas combining DataFrames
In pandas there are **4** ways to combine data from different frames:
* *Merging*
* *Joining*
* *Concatenating*
* *Appending*

Where *Merging* and *Joining* are very similar, and *Concatenating* and *Appending* are also pretty similar. In both cases they are different ways of doing something similar. <br>

So, here, we will go over [Merging](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html "pandas.merge documentation") and [Concatenating](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html "pandas.concat documentation") in Pandas. <br>

Let's get to it!

In [3]:
tips = sns.load_dataset('tips')
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


## Merge
*Merging* is for doing complex column-wise combinations of DataFrames.

In [4]:
tips_bill = tips.groupby(['sex', 'smoker'])[['total_bill', 'tip']].sum()
tips_tip = tips.groupby(['sex', 'smoker'])[['total_bill', 'tip']].sum()

tips_bill.drop('tip', axis=1, inplace=True)
tips_tip.drop('total_bill', axis=1, inplace=True)

In [5]:
tips_bill

total_bill
sex    smoker            
Male   Yes        1337.07
       No         1919.75
Female Yes         593.27
       No          977.68

In [6]:
tips_tip

tip
sex    smoker        
Male   Yes     183.07
       No      302.00
Female Yes      96.74
       No      149.77

Now we combine them by using *Merge* (and pretend they were not combined before)

In [7]:
pd.merge?

In [8]:
# merge on the indexes
pd.merge(
    tips_bill, tips_tip,
    right_index=True, left_index=True
)

total_bill     tip
sex    smoker                    
Male   Yes        1337.07  183.07
       No         1919.75  302.00
Female Yes         593.27   96.74
       No          977.68  149.77

In [9]:
# reset the indexes and then merge on the columns - perhaps the easiest way
pd.merge(
    tips_bill.reset_index(),
    tips_tip.reset_index(), 
    on=['sex', 'smoker']
)

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00
2,Female,Yes,593.27,96.74
3,Female,No,977.68,149.77


In [10]:
# it can figure out which columns are in common by his own
pd.merge(
    tips_bill.reset_index(),
    tips_tip.reset_index()
)

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00
2,Female,Yes,593.27,96.74
3,Female,No,977.68,149.77


In [11]:
# it can merge on partial column and index
pd.merge(
    tips_bill.reset_index(),
    tips_tip,
    left_on=['sex', 'smoker'],
    right_index=True
)

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00
2,Female,Yes,593.27,96.74
3,Female,No,977.68,149.77


In [12]:
pd.merge(
    tips_tip.reset_index(level=0),
    tips_bill.reset_index(),
    on=['sex', 'smoker']
)

,sex,smoker,tip,total_bill
0,Male,Yes,183.07,1337.07
1,Male,No,302.00,1919.75
2,Female,Yes,96.74,593.27
3,Female,No,149.77,977.68


In [13]:
pd.merge(
    tips_bill, tips_tip,
    right_index=True, left_index=True
).rename(index={'Yes': True, 'No': False},
         columns={'total_bill': 'bill'}
)

bill     tip
sex    smoker                 
Male   True    1337.07  183.07
       False   1919.75  302.00
Female True     593.27   96.74
       False    977.68  149.77

In [14]:
# left join will take right side and merge it to the left side.
pd.merge(
    tips_bill.reset_index(),
    tips_tip.reset_index().head(2),
    how='left'
)

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00
2,Female,Yes,593.27,NaN
3,Female,No,977.68,NaN


In [134]:
# right join will merge whatever is on the left side to right side.
pd.merge(
    tips_bill.reset_index().head(2),
    tips_tip.reset_index(),
    how='right'
)

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,183.07
1,Male,No,1919.75,302.00
2,Female,Yes,NaN,96.74
3,Female,No,NaN,149.77


In [16]:
# inner join will not actually do the merge for places where both of these don't exist
pd.merge(
    tips_bill.reset_index().head(2),
    tips_tip.reset_index().tail(3),
    how='inner'
)

,sex,smoker,total_bill,tip
0,Male,No,1919.75,302.0


In [17]:
# outer join
pd.merge(
    tips_bill.reset_index().head(2),
    tips_tip.reset_index().tail(3),
    how='outer'
)

,sex,smoker,total_bill,tip
0,Male,Yes,1337.07,NaN
1,Male,No,1919.75,302.00
2,Female,Yes,NaN,96.74
3,Female,No,NaN,149.77


In [18]:
# If it isn't enough clear, you can add an indicator
pd.merge(
    tips_bill.reset_index().head(3),
    tips_tip.reset_index().tail(3),
    how='outer',
    indicator=True
)

,sex,smoker,total_bill,tip,_merge
0,Male,Yes,1337.07,NaN,left_only
1,Male,No,1919.75,302.00,both
2,Female,Yes,593.27,96.74,both
3,Female,No,NaN,149.77,right_only


In [19]:
# it can handle columns with the same name
pd.merge(
    tips_bill,
    tips_bill,
    right_index=True,
    left_index=True,
    suffixes=('_left', '_right')
)

total_bill_left  total_bill_right
sex    smoker                                   
Male   Yes             1337.07           1337.07
       No              1919.75           1919.75
Female Yes              593.27            593.27
       No               977.68            977.68

## Concatenating
Concatenate can take as many data frames as you want, but it requires that they are specifically contructed. All of the dataframes you pass in will need to have the same index. So no more using columns as index.

In [25]:
# this add data frames together row-wise
pd.concat([tips_bill, tips_bill, tips_tip], sort=False, )

total_bill     tip
sex    smoker                    
Male   Yes        1337.07     NaN
       No         1919.75     NaN
Female Yes         593.27     NaN
       No          977.68     NaN
Male   Yes        1337.07     NaN
       No         1919.75     NaN
Female Yes         593.27     NaN
       No          977.68     NaN
Male   Yes            NaN  183.07
       No             NaN  302.00
Female Yes            NaN   96.74
       No             NaN  149.77

In [60]:
# Column-wise
pd.concat([tips_bill, tips_bill, tips_tip], axis=1)

total_bill  total_bill     tip
sex    smoker                                
Male   Yes        1337.07     1337.07  183.07
       No         1919.75     1919.75  302.00
Female Yes         593.27      593.27   96.74
       No          977.68      977.68  149.77

In [61]:
# This will add on the dataset where it's from
pd.concat([tips_bill, tips_tip], sort=False, keys=['num_0', 'num_1'])

total_bill     tip
      sex    smoker                    
num_0 Male   Yes        1337.07     NaN
             No         1919.75     NaN
      Female Yes         593.27     NaN
             No          977.68     NaN
num_1 Male   Yes            NaN  183.07
             No             NaN  302.00
      Female Yes            NaN   96.74
             No             NaN  149.77

# Exercises

In [80]:
cars1 = pd.read_csv('https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/05_Merge/Auto_MPG/cars1.csv')
cars2 = pd.read_csv('https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/05_Merge/Auto_MPG/cars2.csv')

In [93]:
last_column = cars1.shape[1]

cars1.drop(
    columns=cars1.columns[9:last_column],
    axis=1,
    inplace=True
)

In [95]:
cars1.columns == cars2.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True])

In [96]:
print(cars1.shape)
print(cars2.shape)

(198, 9)
(200, 9)


In [98]:
cars1.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model,origin,car
0,18.0,8,307,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302,140,3449,10.5,70,1,ford torino


In [99]:
cars2.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model,origin,car
0,33.0,4,91,53,1795,17.4,76,3,honda civic
1,20.0,6,225,100,3651,17.7,76,1,dodge aspen se
2,18.0,6,250,78,3574,21.0,76,1,ford granada ghia
3,18.5,6,250,110,3645,16.2,76,1,pontiac ventura sj
4,17.5,6,258,95,3193,17.8,76,1,amc pacer d/l


In [116]:
from tqdm import tqdm

In [117]:
duplicate_cars = []
k = 1

for car1 in tqdm(cars1['car']):
    for car2 in cars2['car']:
        if car1 == car2:
            duplicate_cars.append(car1)
duplicate_cars

100%|█████████████████████████████████████████████████████████████████████████████| 198/198 [00:00<00:00, 16544.59it/s]


['buick estate wagon (sw)',
 'peugeot 504',
 'peugeot 504',
 'toyota corona',
 'ford pinto',
 'ford country squire (sw)',
 'chevrolet malibu',
 'chevrolet caprice classic',
 'chevrolet caprice classic',
 'ford pinto',
 'toyota mark ii',
 'ford pinto',
 'volkswagen dasher',
 'toyota corona',
 'dodge colt',
 'honda civic',
 'honda civic',
 'subaru',
 'buick century',
 'toyota corolla',
 'toyota corolla',
 'toyota corolla',
 'toyota corolla',
 'ford pinto',
 'toyota corona',
 'volkswagen dasher',
 'ford pinto',
 'volkswagen rabbit',
 'peugeot 504',
 'peugeot 504',
 'honda civic cvcc',
 'dodge colt',
 'chevrolet chevette',
 'chevrolet chevette',
 'chevrolet chevette',
 'vw rabbit']

In [115]:
set(duplicate_cars)

{'buick century',
 'buick estate wagon (sw)',
 'chevrolet caprice classic',
 'chevrolet chevette',
 'chevrolet malibu',
 'dodge colt',
 'ford country squire (sw)',
 'ford pinto',
 'honda civic',
 'honda civic cvcc',
 'peugeot 504',
 'subaru',
 'toyota corolla',
 'toyota corona',
 'toyota mark ii',
 'volkswagen dasher',
 'volkswagen rabbit',
 'vw rabbit'}

In [125]:
cars = pd.concat([cars1, cars2]).drop_duplicates()
cars

,mpg,cylinders,displacement,horsepower,weight,acceleration,model,origin,car
0,18.0,8,307,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302,140,3449,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
195,27.0,4,140,86,2790,15.6,82,1,ford mustang gl
196,44.0,4,97,52,2130,24.6,82,2,vw pickup
197,32.0,4,135,84,2295,11.6,82,1,dodge rampage
198,28.0,4,120,79,2625,18.6,82,1,ford ranger


In [129]:
owner_series = pd.Series(np.random.randint(15000, 73000, size=cars.shape[0]))

In [141]:
owner_df = pd.DataFrame(np.random.randint(15000, 73000, size=cars.shape[0]), columns=['owner'])
owner_df

,owner
0,42148
1,63531
2,26060
3,23117
4,28097
...,...
393,52007
394,64609
395,15871
396,22019


In [142]:
pd.merge(
    cars.reset_index(),
    owner_df.reset_index(),
    how='left'
)

,index,mpg,cylinders,displacement,horsepower,weight,acceleration,model,origin,car,owner
0,0,18.0,8,307,130,3504,12.0,70,1,chevrolet chevelle malibu,42148
1,1,15.0,8,350,165,3693,11.5,70,1,buick skylark 320,63531
2,2,18.0,8,318,150,3436,11.0,70,1,plymouth satellite,26060
3,3,16.0,8,304,150,3433,12.0,70,1,amc rebel sst,23117
4,4,17.0,8,302,140,3449,10.5,70,1,ford torino,28097
...,...,...,...,...,...,...,...,...,...,...,...
393,195,27.0,4,140,86,2790,15.6,82,1,ford mustang gl,44112
394,196,44.0,4,97,52,2130,24.6,82,2,vw pickup,44810
395,197,32.0,4,135,84,2295,11.6,82,1,dodge rampage,61446
396,198,28.0,4,120,79,2625,18.6,82,1,ford ranger,31855
